In [ ]:
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options

class Browser:
    _instance = None
    driver: Chrome = None

    def __init__(self) -> None:
        dev_arguments = [
            '--auto-open-devtools-for-tabs',
            '--lang=en'
        ]

        options = Options()

        for argument in dev_arguments:
            options.add_argument(argument)

        self.driver = Chrome(options=options)

    @staticmethod
    def get_instance():
        if Browser._instance is None:
            Browser._instance = Browser()
        return Browser._instance

In [ ]:
import sys
import time
import json
import csv
from typing import Dict
from string import Template

from selenium.common.exceptions import TimeoutException, JavascriptException
from selenium.webdriver import Chrome
from selenium.webdriver import Chrome, ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

import keyring

class Crawler:
    driver: Chrome

    url: str
    pages: Dict[str, str]
    elements: Dict[str, Dict[str, str]]
    scripts: Dict[str, str]

    def __init__(self):
        self.driver = Browser.get_instance().driver
        self.url = 'https://app.tvtime.com'
        self.pages = {
            'welcome': '$url/welcome?mode=auth',
            'sidecar_movies': '$url/sidecar?o=https://msapi.tvtime.com/prod/v1/tracking/cgw/follows/user/$user_id&entity_type=movie&sort=watched_date,desc',
            'sidecar_shows': '$url/sidecar?o=https://api2.tozelabs.com/v2/user/$user_id&fields=shows.fields(id,name,filters,sorting,status,is_followed,is_up_to_date,is_archived,is_for_later,is_favorite).offset(0).limit(500)',
        }
        self.elements = {
            'container': {'xpath': "//flutter-view" },
            'welcome__method': {'offset': [130, -140]},
            'welcome__username': {'offset': [0, 105]},
            'welcome__password': {'offset': [0, 140]},
        }
        self.scripts = {
            'local_storage': self.file_content('./scripts/local_storage.js'),
            'fetch': self.file_content('./scripts/fetch.js')
        }

    def log(self, msg):
        print(msg, file=sys.stdout)

    def error(self, msg):
        print(msg, file=sys.stderr)

    def save_json_file(self, file, content):
        with open(file, 'w', encoding='utf-8') as f:
            json.dump(content, f, ensure_ascii=False, indent=2)

    def save_csv_file(self, file, content, fieldnames):
        with open(file, 'w', encoding='utf8', newline='') as file:
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(content)

    def file_content(self, file):
        with open(file, 'r', encoding='utf-8') as f:
            return f.read()

    def username(self):
        username = keyring.get_password('tvtime', 'username')

        if username is None:
            username = input('Insert your username or email:')
            keyring.set_password('tvtime', 'username', username)

        return username

    def password(self):
        password = keyring.get_password('tvtime', 'password')

        if password is None:
            password = input('Insert your password:')
            keyring.set_password('tvtime', 'password', password)

        return password

    def credentails(self):
        username, password = self.username(), self.password()
        return username, password

    def arrangements(self):
        self.driver.implicitly_wait(5)

    def wait(self, seconds = 10):
        time.sleep(seconds)

    def focus(self):
        self.driver.switch_to.window(self.driver.current_window_handle)

    def page(self, url_key, **url_arguments):
        url = Template(self.pages.get(url_key)).substitute(url=self.url, **url_arguments)
        self.driver.get(url)

    def local_storage(self, *args):
        script_key = 'local_storage'
        script = self.scripts.get(script_key)
        try:
            return json.loads(self.driver.execute_script(script, *args))
        except JavascriptException as err:
            self.error(err)
            return None

    def fetch(self, url_key):
        script_key = 'fetch'
        script = self.scripts.get(script_key)
        user_id = user_id = crawler.local_storage('flutter.user').get('id')
        url = Template(self.pages.get(url_key)).substitute(url=self.url, user_id=user_id)
        try:
            return self.driver.execute_async_script(script, url)
        except JavascriptException as err:
            self.error(err)
            return None

    def element(self, element_key):
        xpath = self.elements.get(element_key).get('xpath')
        try:
            return WebDriverWait(self.driver, 30)\
                .until(EC.presence_of_element_located((By.XPATH, xpath)))
        except TimeoutException as err:
            self.error(err)
            return None

    def guess_position(self, element_key):
        container = self.element('container')
        x, y = self.elements.get(element_key).get('offset')
        height = container.rect.get('height')
        y = (height + y - (height / 2)) if y < 0 else (y - (height / 2))
        return container, x, y

    def build_actions(self, element_key):
        actions = ActionChains(self.driver)
        container, x, y = self.guess_position(element_key)
        return actions, container, x, y

    def click(self, element_key):
        actions, container, x, y = self.build_actions(element_key)
        actions.move_to_element_with_offset(container, x, y)\
            .click().perform()

    def type(self, element_key, value):
        actions, container, x, y = self.build_actions(element_key)
        actions.move_to_element_with_offset(container, x, y)\
            .click().send_keys(value).send_keys(Keys.ENTER).perform()

crawler = Crawler()

In [ ]:
username, password = crawler.credentails()

crawler.arrangements()
crawler.focus()

crawler.page('welcome'), crawler.wait()

crawler.click('welcome__method'), crawler.wait()
crawler.type('welcome__username', username), crawler.wait()
crawler.type('welcome__password', password)

In [ ]:
api_movies = crawler.fetch('sidecar_movies')
crawler.save_json_file('./exported/movies.json', api_movies)

def map_movies(movie):
    meta = movie.get('meta')
    return {
        'Name': meta.get('name'),
        'Year': meta.get('first_release_date').split('-')[0],
    }

movies = list(map(map_movies, api_movies.get('data').get('objects')))
movies.sort(key=lambda movie: movie['Year'])
crawler.save_csv_file('./exported/movies.csv', movies, fieldnames=['Name', 'Year'])


In [ ]:
api_shows = crawler.fetch('sidecar_shows')
crawler.save_json_file('./exported/shows.json', api_shows)

def map_shows(show):
    return {
        'Name': show.get('name')
    }

shows = list(map(map_shows, api_shows.get('shows')))
crawler.save_csv_file('./exported/shows.csv', shows, fieldnames=['Name'])